In [ ]:
from IPython.core.display import HTML
css_file = './custom.css'
HTML(open(css_file, "r").read())

###### Content provided under a Creative Commons Attribution license, CC-BY 4.0; code under MIT License. (c)2015-2023 [David I. Ketcheson](http://davidketcheson.info)

##### Version 0.4 - March 2023

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)

fft = np.fft.fft
ifft = np.fft.ifft

In [ ]:
def plot_solution(frames, uuhat, x, tt, xi):
    fig = plt.figure(figsize=(12,8))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x,frames[0],lw=3)
    line2, = axes2.semilogy(xi,np.abs(np.fft.fft(frames[0])))
    xi_max = np.max(np.abs(xi))
    axes2.semilogy([xi_max/2.,xi_max/2.],[1.e-6,4e8],'--r')
    axes2.semilogy([-xi_max/2.,-xi_max/2.],[1.e-8,4e10],'--r')
    axes.set_xlabel(r'$x$',fontsize=30)
    axes2.set_xlabel(r'$\xi$',fontsize=30)
    plt.tight_layout()
    plt.close()

    def plot_frame(i):
        line.set_data(x,frames[i])
        power_spectrum = np.abs(uuhat[i])**2
        line2.set_data(np.sort(xi),power_spectrum[np.argsort(xi)])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((-np.pi,np.pi))
        axes.set_ylim((-200,3000))

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())

# Exponential methods

As we have seen, the exact solution of a linear PDE can be obtained simply by applying the corresponding exponential operator.  We took advantage of this in the operator splitting approach above in order to solve the linear part of the problem exactly.  It's natural to ask if the idea of using the exponential operator can be incorporated directly into a linear multistep or Runge-Kutta type method, in order to avoid the splitting error.  This leads to the concept of exponential integrators.

A natural way to derive many exponential integrators starts from Duhamel's principle (also known as the variation-of-constants formula:

$$
u(t_n+\Delta t) = e^{\Delta t L}u(t_n) + \int_0^{\Delta t} e^{(\Delta t-\tau) L} f(u(t_n+\tau)).
$$

By choosing different discrete approximations of the integral, different exponential methods are obtained.  One of the simplest is the first-order **exponential Euler** method:

$$
    u^{n+1} = e^{\Delta t L}u^n + \Delta t \phi_1(\Delta t L) f(u^n).
$$

Here $\phi_1(z)=(e^z-1)/z$.  This method is first-order accurate.  Like the explicit Euler method, it is not typically suitable for wave equations because of its stability properties.

## Exponential Euler implementation
This one is linearly unstable because Euler's method doesn't contain any part of the imaginary axis.  But if we take a small enough timestep, the solution looks reasonable for some time.

In [ ]:
def solve_KdV_Exp1(m,dt):
    L = 2*np.pi
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    u = 1500*np.exp(-10*(x+2)**2)
    tmax = 0.005

    uhat2 = np.abs(np.fft.fft(u))

    num_plots = 50
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    frames = [u.copy()]
    tt = [0]
    uuhat = [uhat2]
    filtr = np.ones_like(u)

    D = 1j*xi**3*dt
    eps = 1.e-16
    phi1 = (np.exp(D)-1.)/(1j*xi**3*dt+eps)
    phi1 = np.diag(phi1)
    
    for n in range(1,nmax+1):
        uhat = np.fft.fft(u)
        uux = -u*np.real(np.fft.ifft(1j*xi*uhat*filtr))
        uuxhat = np.fft.fft(uux)
        u_new = np.real(np.fft.ifft(np.exp(1j*xi**3*dt)*uhat) \
                    + dt*(np.fft.ifft(phi1@uuxhat)))
        
        u = u_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(u.copy())
            tt.append(t)
            uhat2 = np.abs(np.fft.fft(u))
            uuhat.append(uhat2)
    return frames, uuhat, x, tt, xi

In [ ]:
m = 512
umax = 1500
dt = 1.73/(m/2)/umax * 0.1

frames, uuhat, x, tt, xi = solve_KdV_Exp1(m,dt)

In [ ]:
plot_solution(frames, uuhat, x, tt, xi)

Here we see that, even though we've taken a relatively small step size, the solution grows over time.

Higher-order exponential Runge-Kutta methods can be constructed using additional stages and higher order $\phi$ functions, which are defined recursively by
$\phi_{k+1}(z) = (\phi_k(z)-\phi_k(0))/z$.  The accurate evaluation of these functions for both large and small values of $z$ requires special techniques, such as those employed by [Kassam & Trefethen](https://epubs.siam.org/doi/epdf/10.1137/S1064827502410633).

## 4th-order exponential (Lawson4) method

Alternatively, one can design methods that use only the exponential function itself.  Such methods are often referred to as Lawson-type methods.  Here we implement a 4th-order Lawson-type method that is similar to a Runge-Kutta method:

\begin{align}
y_i & = \exp(c_i \Delta t L)u^n + h \sum_{j=1}^s a_{ij}(\Delta t L)f(y_j) \\
u^{n+1} & = \exp(\Delta t L)u^n + h \sum_{j=1}^s b_j(\Delta t L)f(y_j).
\end{align}

The coefficients are functions of the linear operator $\Delta t L$.  Starting from a traditional Runge-Kutta method, the coefficients of a Lawson method are simply

\begin{align}
    a_{ij}(z) & = \tilde{a}_{ij}(z) \exp((c_i-c_j)z) \\
    b_j(z) & = \tilde{b}_j(z) \exp((1-c_j)z).
\end{align}

The method implemented below is based on the classical 4th-order Runge-Kutta method, and has the following coefficients:

\begin{align}
\begin{array}{c|cccc}
 &  &  &  & \\
\frac{1}{2} & \frac{1}{2}e^{z/2} &  &  & \\
\frac{1}{2} &  & \frac{1}{2} &  & \\
1 &  &  & e^{z/2} & \\
\hline
 & \frac{1}{6}e^z & \frac{1}{3}e^{z/2} & \frac{1}{3}e^{z/2} & \frac{1}{6}\\
\end{array}
\end{align}

In [ ]:
def uux(u,xi,filtr):
    return -u*np.real(ifft(1j*xi*fft(u)*filtr))

def solve_KdV_Exp4(m,dt,use_filter=True):
    L = 2*np.pi
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    u = 1500*np.exp(-10*(x+2)**2)
    tmax = 0.005

    uhat2 = np.abs(np.fft.fft(u))

    num_plots = 50
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    frames = [u.copy()]
    tt = [0]
    uuhat = [uhat2]
    filtr = np.ones_like(u)
    if use_filter:
        xi_max = np.max(np.abs(xi))
        filtr[np.where(np.abs(xi)>xi_max*2./3)] = 0.
        
    z = 1j*xi**3*dt
    ez2 = np.exp(0.5*z)
    ez = np.exp(z)
    
    for n in range(1,nmax+1):
        uhat = fft(u)
        
        y2 = ifft(ez2*uhat) + dt/2 * ifft(ez2*fft(uux(u,xi,filtr)))
        y3 = ifft(ez2*uhat) + dt/2 * uux(y2,xi,filtr)
        y4 = ifft(ez*uhat)  + dt   * ifft(ez2*fft(uux(y3,xi,filtr)))
        u_new = np.real(ifft(ez*uhat) + dt/6 * (ifft(ez*fft(uux(u,xi,filtr))) \
                    + 2*ifft(ez2*fft(uux(y2,xi,filtr))) \
                    + 2*ifft(ez2*fft(uux(y3,xi,filtr))) \
                    + uux(y4,xi,filtr) ))
        u_new = ifft(filtr*fft(u_new))
        
        u = u_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(u.copy())
            tt.append(t)
            uhat2 = np.abs(np.fft.fft(u))
            uuhat.append(uhat2)
    return frames, uuhat, x, tt, xi

In [ ]:
m = 2048
umax = 3000
dt = 1./(m/2)/umax

frames, uuhat, x, tt, xi = solve_KdV_Exp4(m,dt,True)

In [ ]:
plot_solution(frames, uuhat, x, tt, xi)

The implementation here is not optimal and could be made even faster.  Nevertheless, we see that a highly accurate solution can be obtained in a fraction of the time required for our original explicit RK implementation.  We see that filtering is required, and even with filtering the solution becomes unstable after sufficiently long times.

## Krasny filtering
As an alternative filtering approach, one can simply remove all energy from Fourier modes that already have only a small amount of energy.  This is implemented below.

In [ ]:
fft = np.fft.fft
ifft = np.fft.ifft

def uux(u,xi,filtr):
    return -u*np.real(ifft(1j*xi*fft(u)))

def solve_KdV_Exp4(m,dt,use_filter="Krasny"):
    L = 2*np.pi
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    u = 1500*np.exp(-10*(x+2)**2)
    tmax = 0.005

    uhat2 = np.abs(np.fft.fft(u))

    num_plots = 50
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    frames = [u.copy()]
    tt = [0]
    uuhat = [uhat2]
        
    z = 1j*xi**3*dt
    ez2 = np.exp(0.5*z)
    ez = np.exp(z)
    filtr = np.ones_like(u)
    
    for n in range(1,nmax+1):
        uhat = fft(u)
        
        y2 = ifft(ez2*uhat) + dt/2 * ifft(ez2*fft(uux(u,xi,filtr)))
        y3 = ifft(ez2*uhat) + dt/2 * uux(y2,xi,filtr)
        y4 = ifft(ez*uhat)  + dt   * ifft(ez2*fft(uux(y3,xi,filtr)))
        u_new = np.real(ifft(ez*uhat) + dt/6 * (ifft(ez*fft(uux(u,xi,filtr))) \
                    + 2*ifft(ez2*fft(uux(y2,xi,filtr))) \
                    + 2*ifft(ez2*fft(uux(y3,xi,filtr))) \
                    + uux(y4,xi,filtr) ))
        if use_filter == "Krasny":
            power = np.abs(fft(u_new))
            maxpow = np.max(power)
            filtr = np.ones_like(u)
            filtr[np.where(power/maxpow<1.e-8)] = 0.
            u_new = np.real(ifft(filtr*fft(u_new)))
        
        u = u_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(u.copy())
            tt.append(t)
            uhat2 = np.abs(np.fft.fft(u))
            uuhat.append(uhat2)
    return frames, uuhat, x, tt, xi

In [ ]:
m = 2048
umax = 1500
dt = 1./(m/2)/umax

frames, uuhat, x, tt, xi = solve_KdV_Exp4(m,dt,"Krasny")

In [ ]:
plot_solution(frames, uuhat, x, tt, xi)

The solution with Krasny filtering seems to be more stable overall, and doesn't blow up even after long times.  The downside of this approach is that it requires a cutoff parameter.  If this parameter is set too small, the solution may be unstable, while if it is see too large then accuracy may be lost.

# References

There are a couple of somewhat-outdated but still useful review papers on this topic:

- [Minchev & Wright](https://cds.cern.ch/record/848122/files/cer-002531456.pdf)
- [Hochbruck & Ostermann](https://doi.org/10.1017/S0962492910000048)

The development of this class methods continues to be a very active area of research.